In [3]:
import apache_beam as beam
import glob
import os
import sys
import string

from datetime import datetime
from apache_beam.options import pipeline_options
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.runners import DataflowRunner

import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner

In [4]:
! gsutil -m cp -r gs://text-analysis-323506/data ./
! unzip ./data/data_archive.zip 
! bzip2 -d train.ft.txt.bz2 
! bzip2 -d test.ft.txt.bz2
! mv *.ft.txt ./data/

unzip:  cannot find or open ./data/data_archive.zip, ./data/data_archive.zip.zip or ./data/data_archive.zip.ZIP.
bzip2: Can't open input file train.ft.txt.bz2: No such file or directory.
bzip2: Can't open input file test.ft.txt.bz2: No such file or directory.
mv: cannot stat '*.ft.txt': No such file or directory


In [15]:
def contract_line(line: str):
    contraction_dict = {"ain't": "are not", "'s":" is", "aren't": "are not", "don't": "do not", "didn't": "did not", "won't": "will not", 
                   "can't": "cannot"}
    
    words = line.split()
    for i in range(len(words)):
        if words[i] in contraction_dict:
            words[i] = contraction_dict[words[i]]
    return ' '.join(words)

In [16]:
def to_lower_case(line: str):
    return line.lower()

In [17]:
def remove_punctuations(line: str):
    return line.translate(str.maketrans('', '', string.punctuation))

In [18]:
def remove_stopwords(line: str):
    from nltk.corpus import stopwords
    
    stop_words = set(stopwords.words('english'))
    return " ".join([word for word in line.split() if word not in stop_words])

In [21]:
def remove_special_chars(line: str):
    import re
    return re.sub('[\W\_]','',line)

In [2]:
def remove_emojis(line: str):
    import re
    
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', line)